In [2]:
# -*- coding: utf-8 -*-
# Импорты
CUDA_VISIBLE_DEVICES=-1 
import os
import pandas as pd
from multiprocessing import Pool
import warnings
warnings.filterwarnings("ignore")
import numpy as np


In [3]:
def listdir_fullpath(d):
    return [os.path.join(d, f) for f in sorted(os.listdir(d))]
   
   # Функция загрузки одного файла
dateparse = lambda x: pd.datetime.strptime(x, '%d.%m.%Y %H:%M:%S.%f')
def load(fname):
    R = pd.read_hdf(fname)
    # Выделяем сделки для построения OHLC
    D = R[R.DealPrice > 0].drop(['Price', 'Flags'], 1)
    T = D.resample(Frame, how={'DealPrice': 'ohlc'})
    T['Change'] = T.DealPrice.close - T.DealPrice.open
    
    # Делим на buy/sell-факторы
    for b in BS:
        S = R[R.Flags.str.contains(b, regex=False)]
        for s in SA:
            F = S[S.Flags.str.contains(s, regex=False)].resample(Frame)
            T = T.join(F.count().rename(columns = {'Amount': s + b + 'Cnt'}).drop(['Price', 'DealPrice', 'OI', 'Flags'], 1))
            T = T.join(F.sum().rename(columns = {'Amount': s + b + 'Sum'}).drop(['Price', 'DealPrice', 'OI'], 1))
    
    for s in SA:
        T[s + 'CntDelta'] = T[s + 'BuyCnt'] - T[s + 'SellCnt']
        T[s + 'CntRatio'] = T[s + 'BuyCnt'] / T[s + 'SellCnt'] - 1
        T[s + 'SumDelta'] = T[s + 'BuySum'] - T[s + 'SellSum']
        T[s + 'SumRatio'] = T[s + 'BuySum'] / T[s + 'SellSum'] - 1
        
    return T

In [6]:
# Параметры
Frame='60s'
prefix = '../../hdf/'
postfix = ['RTS-12.15']
dirr = [prefix+x for x in postfix]
BS = ['Buy', 'Sell']
SA = ['Add', 'Fill', 'Cancel', 'Move']

# Загружаем все файлы параллельно, затем соединяем их в один DataFrame
R = pd.DataFrame()
for t in dirr:
    R1 = pd.concat(Pool().map(load, listdir_fullpath(t)))
    R = pd.concat([R, R1])

# Дропнем nan'ы
R = R.dropna()
R = R.replace([np.inf, -np.inf], np.nan).dropna(how="all")

In [7]:
R.to_pickle(str(postfix))  # where to save it, usually as a .pkl